In [1]:
import pandas as pd
import requests
import credentials
client_secret = credentials.ClientSecretCO
client_id = credentials.ClientIDCO

def get_acces_token(client_id, client_secret):
    """retourne un json (dict)"""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": f"{client_id}",
        f"client_secret": f"{client_secret}"
    }

    response = requests.post(url, headers=headers, data=data)

    return response.json()

accessToken = get_acces_token(client_id, client_secret)["access_token"]

def get_playlist(accessToken, playlist_id):
    playlist = requests.get(
        url="https://api.spotify.com/v1/playlists/" + playlist_id,
        headers={
            "Authorization": 'Bearer ' + accessToken
        }
    )
    return playlist.json() 

def get_several_tracks_features(accessToken, songs_id : list) -> list:
    """maximum 100 ids"""
    if (len(songs_id) > 100):
        raise ValueError("Too Many Songs")
    songs_id_string = ",".join(songs_id)
    songs_features = requests.get(
        url="https://api.spotify.com/v1/audio-features?ids=" + songs_id_string,
        headers={
            "Authorization": 'Bearer ' + accessToken
        }
    )
    return songs_features.json()

def get_playlist_tracks_features(accessToken, playlist, country): 
    """params: spotify playlist json object, returns a pandas dataframe"""
    songs_id = []
    songs_titles = []
    songs_artists = []
    for track in playlist["tracks"]["items"]:
        info = track["track"]
        songs_titles.append(info["name"])
        songs_artists.append([artist["id"] for artist in info["artists"]])
        songs_id.append(info["id"])
    songs_features_raw = get_several_tracks_features(accessToken, songs_id)
    songs_features = songs_features_raw["audio_features"]
    #danceabilities = [song["danceability"] for song in songs_features]

    if len(songs_features) != len(songs_id):
        print("Not all songs have been succesfully analysed, aborted")

    else :
        temp = pd.DataFrame({"title" : songs_titles, "artists_id": songs_artists})
        songs_df = pd.DataFrame(songs_features)
        songs_df = pd.concat([temp, songs_df], axis = 1)
        songs_df["country"] = country
        return songs_df

def get_top50_features(playlists_df, accessToken):
    features_df = pd.DataFrame()
    for ind in playlists_df.index:
        playlist_id = playlists_df["playlist_id"][ind]
        country = playlists_df["target_country"][ind]
        playlist = get_playlist(accessToken, playlist_id)
        new_playlist_df = get_playlist_tracks_features(accessToken, playlist, country)
        features_df = pd.concat([features_df, new_playlist_df], axis=0)
    return features_df


Date d'extraction : 11/11/2024

In [9]:
playlist_df = pd.read_csv("./data/cleaned_top50_playlists_data.csv")
features_df = get_top50_features(playlist_df, accessToken)

features_df

,title,artists_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,country
0,IA,[78F6Xkd46aMHkr4AP60TXY],0.706,0.631,0,-5.812,0,0.0366,0.6560,0.000000,...,0.289,111.930,audio_features,7lcbpfihslisNObKdHjyLo,spotify:track:7lcbpfihslisNObKdHjyLo,https://api.spotify.com/v1/tracks/7lcbpfihslis...,https://api.spotify.com/v1/audio-analysis/7lcb...,171429,4,Andorra
1,Q U E V A S H A C E R H O Y ?,"[3E12tRURRvPfHz0hAMCFYc, 54seKvtsZauR1iauN0ptpo]",0.560,0.681,6,-6.304,0,0.2440,0.1030,0.000000,...,0.388,155.079,audio_features,0Sr7ssScx54yxdM2ols7nW,spotify:track:0Sr7ssScx54yxdM2ols7nW,https://api.spotify.com/v1/tracks/0Sr7ssScx54y...,https://api.spotify.com/v1/audio-analysis/0Sr7...,223320,4,Andorra
2,DIABLA,[30Ml7nTMkXYyb0UoLwtmCc],0.543,0.587,10,-7.709,0,0.4970,0.7860,0.000000,...,0.723,174.075,audio_features,00EWeXw1RLUwCMw8pxtXAE,spotify:track:00EWeXw1RLUwCMw8pxtXAE,https://api.spotify.com/v1/tracks/00EWeXw1RLUw...,https://api.spotify.com/v1/audio-analysis/00EW...,217946,4,Andorra
3,DEGENERE 👑 (feat. benny blanco),"[7iK8PXO48WeuP03g8YR51W, 5CiGnKThu5ctn9pBxv7DGa]",0.744,0.709,6,-5.778,0,0.2840,0.0501,0.000000,...,0.712,194.055,audio_features,0qeIFEGRQqHoaJtLCl0U73,spotify:track:0qeIFEGRQqHoaJtLCl0U73,https://api.spotify.com/v1/tracks/0qeIFEGRQqHo...,https://api.spotify.com/v1/audio-analysis/0qeI...,130880,4,Andorra
4,Me has invitado a bailar,[0CVOcYvRcSvOXyuR4YGKaC],0.665,0.544,10,-3.648,1,0.1150,0.0129,0.000000,...,0.625,163.948,audio_features,3tMftGMXTPbQlmjh4FzXpZ,spotify:track:3tMftGMXTPbQlmjh4FzXpZ,https://api.spotify.com/v1/tracks/3tMftGMXTPbQ...,https://api.spotify.com/v1/audio-analysis/3tMf...,192906,4,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,shhhhhhh..,"[30eFAXoU2kTjJPf2cq80B8, 3diftVOq7aEIebXKkC34oR]",0.739,0.537,2,-7.241,0,0.0394,0.2510,0.000000,...,0.379,137.970,audio_features,4jWmhzf53Q0ZvZMuGdiR9x,spotify:track:4jWmhzf53Q0ZvZMuGdiR9x,https://api.spotify.com/v1/tracks/4jWmhzf53Q0Z...,https://api.spotify.com/v1/audio-analysis/4jWm...,230676,4,Vietnam
46,Simp Gái 808,[6TITnFVRcl0AcZ4syE7Toe],0.778,0.514,8,-12.450,0,0.1970,0.3550,0.000535,...,0.224,94.994,audio_features,5J3m0taZ0W1E9HTE0MwcSv,spotify:track:5J3m0taZ0W1E9HTE0MwcSv,https://api.spotify.com/v1/tracks/5J3m0taZ0W1E...,https://api.spotify.com/v1/audio-analysis/5J3m...,161948,4,Vietnam
47,Say Yes (Vietnamese Version),"[61ydMyohF67xBZmWSPA3YQ, 5mPbBOU2KMFfStepNZ027...",0.679,0.747,4,-6.917,1,0.0575,0.2860,0.000000,...,0.505,100.073,audio_features,7mbWwlHgukuuW8r7Nc3FHv,spotify:track:7mbWwlHgukuuW8r7Nc3FHv,https://api.spotify.com/v1/tracks/7mbWwlHgukuu...,https://api.spotify.com/v1/audio-analysis/7mbW...,223200,4,Vietnam
48,"REGRET (feat. Quân A.P, Pháp Kiều, Quang Trung...","[4wZ6awunqaaVLl5j0WpDFu, 2NRcG7E1j2sSi8vnUzCcp...",0.548,0.636,8,-7.157,1,0.0551,0.1310,0.000000,...,0.114,155.889,audio_features,3JIfmjJbanxJkwaPzYa2gN,spotify:track:3JIfmjJbanxJkwaPzYa2gN,https://api.spotify.com/v1/tracks/3JIfmjJbanxJ...,https://api.spotify.com/v1/audio-analysis/3JIf...,252308,4,Vietnam


In [1]:
usa_playlist = get_playlist(accessToken, "37i9dQZEVXbLRQDuF5jeBp")
usa_features = get_playlist_tracks_features(accessToken, usa_playlist, "United States")
usa_features

NameError: name 'get_playlist' is not defined

In [2]:
#top_50_features = pd.read_csv("./data/top50_features_11_11_24.csv")
#top50_features_avec_USA = pd.concat([usa_features, top_50_features])
usa_features

NameError: name 'usa_features' is not defined

In [7]:
top50_features_avec_USA

,title,artists_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,country,index
0,squabble up,[2YZyLoL8N0Wb9xBt1NhZWg],0.874,0.672,0,-5.568,1,0.1980,0.02060,0.000000,...,103.921,audio_features,0nj9Bq5sHDiTxSHunhgkFb,spotify:track:0nj9Bq5sHDiTxSHunhgkFb,https://api.spotify.com/v1/tracks/0nj9Bq5sHDiT...,https://api.spotify.com/v1/audio-analysis/0nj9...,157992,4,United States,NaN
1,tv off (feat. lefty gunplay),"[2YZyLoL8N0Wb9xBt1NhZWg, 1jiZvw42D4oquLl24x2VWV]",0.855,0.529,6,-6.679,0,0.2630,0.08370,0.000000,...,100.036,audio_features,0aB0v4027ukVziUGwVGYpG,spotify:track:0aB0v4027ukVziUGwVGYpG,https://api.spotify.com/v1/tracks/0aB0v4027ukV...,https://api.spotify.com/v1/audio-analysis/0aB0...,220690,4,United States,NaN
2,luther (with sza),"[2YZyLoL8N0Wb9xBt1NhZWg, 7tYKF4w9nC0nq9CsPZTHyP]",0.707,0.575,2,-7.546,1,0.1250,0.25100,0.000000,...,138.008,audio_features,2CGNAOSuO1MEFCbBRgUzjd,spotify:track:2CGNAOSuO1MEFCbBRgUzjd,https://api.spotify.com/v1/tracks/2CGNAOSuO1ME...,https://api.spotify.com/v1/audio-analysis/2CGN...,177599,4,United States,NaN
3,wacced out murals,[2YZyLoL8N0Wb9xBt1NhZWg],0.761,0.501,9,-10.759,0,0.4590,0.24900,0.000000,...,81.998,audio_features,5gOfC9UzZQzTyShqPMrpjT,spotify:track:5gOfC9UzZQzTyShqPMrpjT,https://api.spotify.com/v1/tracks/5gOfC9UzZQzT...,https://api.spotify.com/v1/audio-analysis/5gOf...,317093,4,United States,NaN
4,hey now (feat. dody6),"[2YZyLoL8N0Wb9xBt1NhZWg, 4VHa48wXlsDA2vWfgIi7cX]",0.806,0.358,10,-8.769,1,0.1170,0.00443,0.001620,...,102.011,audio_features,5S8VwnB4sLi6W0lYTWYylu,spotify:track:5S8VwnB4sLi6W0lYTWYylu,https://api.spotify.com/v1/tracks/5S8VwnB4sLi6...,https://api.spotify.com/v1/audio-analysis/5S8V...,217479,4,United States,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,shhhhhhh..,"[30eFAXoU2kTjJPf2cq80B8, 3diftVOq7aEIebXKkC34oR]",0.739,0.537,2,-7.241,0,0.0394,0.25100,0.000000,...,137.970,audio_features,4jWmhzf53Q0ZvZMuGdiR9x,spotify:track:4jWmhzf53Q0ZvZMuGdiR9x,https://api.spotify.com/v1/tracks/4jWmhzf53Q0Z...,https://api.spotify.com/v1/audio-analysis/4jWm...,230676,4,Vietnam,45.0
3396,Simp Gái 808,[6TITnFVRcl0AcZ4syE7Toe],0.778,0.514,8,-12.450,0,0.1970,0.35500,0.000535,...,94.994,audio_features,5J3m0taZ0W1E9HTE0MwcSv,spotify:track:5J3m0taZ0W1E9HTE0MwcSv,https://api.spotify.com/v1/tracks/5J3m0taZ0W1E...,https://api.spotify.com/v1/audio-analysis/5J3m...,161948,4,Vietnam,46.0
3397,Say Yes (Vietnamese Version),"[61ydMyohF67xBZmWSPA3YQ, 5mPbBOU2KMFfStepNZ027...",0.679,0.747,4,-6.917,1,0.0575,0.28600,0.000000,...,100.073,audio_features,7mbWwlHgukuuW8r7Nc3FHv,spotify:track:7mbWwlHgukuuW8r7Nc3FHv,https://api.spotify.com/v1/tracks/7mbWwlHgukuu...,https://api.spotify.com/v1/audio-analysis/7mbW...,223200,4,Vietnam,47.0
3398,"REGRET (feat. Quân A.P, Pháp Kiều, Quang Trung...","[4wZ6awunqaaVLl5j0WpDFu, 2NRcG7E1j2sSi8vnUzCcp...",0.548,0.636,8,-7.157,1,0.0551,0.13100,0.000000,...,155.889,audio_features,3JIfmjJbanxJkwaPzYa2gN,spotify:track:3JIfmjJbanxJkwaPzYa2gN,https://api.spotify.com/v1/tracks/3JIfmjJbanxJ...,https://api.spotify.com/v1/audio-analysis/3JIf...,252308,4,Vietnam,48.0


In [10]:
# ajout des features de l'usa dans la bonne catégories

top_50_features_north_america = pd.read_csv("./data/North_America.csv")
north_america_avec_USA = pd.concat([usa_features, top_50_features_north_america])
top_50_features_north_america

,id,title,artists_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,type,id.1,uri,track_href,analysis_url,duration_ms,time_signature,country,Unnamed: 22,Unnamed: 23
0,0,That’s So True,[4tuJ0bMpJh08umKkEXKUI5],0.554,0.808,1,-4.169,1,0.0368,0.2140,...,audio_features,7ne4VBA60CxGM75vw0EYad,spotify:track:7ne4VBA60CxGM75vw0EYad,https://api.spotify.com/v1/tracks/7ne4VBA60CxG...,https://api.spotify.com/v1/audio-analysis/7ne4...,166300,4,Canada,NaN,NaN
1,1,APT.,"[3eVa5w3URK5duf6eyVDbu9, 0du5cEVh5yTK9QJze8zA0C]",0.777,0.783,0,-4.477,0,0.2600,0.0283,...,audio_features,5vNRhkKd0yEAg8suGBpjeY,spotify:track:5vNRhkKd0yEAg8suGBpjeY,https://api.spotify.com/v1/tracks/5vNRhkKd0yEA...,https://api.spotify.com/v1/audio-analysis/5vNR...,169917,4,Canada,NaN,NaN
2,2,Die With A Smile,"[1HY2Jd0NmPuamShAr6KMms, 0du5cEVh5yTK9QJze8zA0C]",0.521,0.592,6,-7.777,0,0.0304,0.3080,...,audio_features,2plbrEY59IikOBgBGLjaoe,spotify:track:2plbrEY59IikOBgBGLjaoe,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,https://api.spotify.com/v1/audio-analysis/2plb...,251668,3,Canada,NaN,NaN
3,3,A Bar Song (Tipsy),[3y2cIKLjiOlp1Np37WiUdH],0.722,0.706,9,-4.857,1,0.0292,0.0720,...,audio_features,5fZJQrFKWQLb7FpJXZ1g7K,spotify:track:5fZJQrFKWQLb7FpJXZ1g7K,https://api.spotify.com/v1/tracks/5fZJQrFKWQLb...,https://api.spotify.com/v1/audio-analysis/5fZJ...,171292,4,Canada,NaN,NaN
4,4,Sailor Song,[1iCnM8foFssWlPRLfAbIwo],0.494,0.413,11,-10.432,1,0.0254,0.6820,...,audio_features,2262bWmqomIaJXwCRHr13j,spotify:track:2262bWmqomIaJXwCRHr13j,https://api.spotify.com/v1/tracks/2262bWmqomIa...,https://api.spotify.com/v1/audio-analysis/2262...,211979,4,Canada,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,45,Imagínate,"[5H1nN1SzW0qNeUEZvuXjAj, 3UTF2no3muGdiFXVujl94i]",0.779,0.737,11,-4.667,1,0.1090,0.1340,...,audio_features,4zbEItKoaRId1vRZkoO0Uh,spotify:track:4zbEItKoaRId1vRZkoO0Uh,https://api.spotify.com/v1/tracks/4zbEItKoaRId...,https://api.spotify.com/v1/audio-analysis/4zbE...,170366,4,Panama,NaN,NaN
396,46,Tu Chacal,"[5A0VwTrVvV03LMeBZYT99d, 71DQO8pgnTwS3lNCq0cDc...",0.660,0.679,8,-6.010,0,0.3630,0.3360,...,audio_features,5IWsYLhoYS1kTaMh2bVnhz,spotify:track:5IWsYLhoYS1kTaMh2bVnhz,https://api.spotify.com/v1/tracks/5IWsYLhoYS1k...,https://api.spotify.com/v1/audio-analysis/5IWs...,171812,5,Panama,NaN,NaN
397,47,Tu Man,"[4I8dxVJ518Fc8xqfIilTeU, 71DQO8pgnTwS3lNCq0cDcr]",0.506,0.534,4,-7.153,0,0.0765,0.7300,...,audio_features,3wMxU6KYZ89HhYSBoJe78M,spotify:track:3wMxU6KYZ89HhYSBoJe78M,https://api.spotify.com/v1/tracks/3wMxU6KYZ89H...,https://api.spotify.com/v1/audio-analysis/3wMx...,237167,4,Panama,NaN,NaN
398,48,Yo Te Llamo,"[0ghH5fsPlkf2XIALKJNSsb, 5A0VwTrVvV03LMeBZYT99d]",0.614,0.646,5,-7.086,1,0.3600,0.1270,...,audio_features,4lFnfbRhGSpTm17XKGcXs6,spotify:track:4lFnfbRhGSpTm17XKGcXs6,https://api.spotify.com/v1/tracks/4lFnfbRhGSpT...,https://api.spotify.com/v1/audio-analysis/4lFn...,153900,4,Panama,NaN,NaN


In [11]:
usa_features

,title,artists_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,country
0,squabble up,[2YZyLoL8N0Wb9xBt1NhZWg],0.874,0.672,0,-5.568,1,0.1980,0.02060,0.000000,...,0.711,103.921,audio_features,0nj9Bq5sHDiTxSHunhgkFb,spotify:track:0nj9Bq5sHDiTxSHunhgkFb,https://api.spotify.com/v1/tracks/0nj9Bq5sHDiT...,https://api.spotify.com/v1/audio-analysis/0nj9...,157992,4,United States
1,tv off (feat. lefty gunplay),"[2YZyLoL8N0Wb9xBt1NhZWg, 1jiZvw42D4oquLl24x2VWV]",0.855,0.529,6,-6.679,0,0.2630,0.08370,0.000000,...,0.548,100.036,audio_features,0aB0v4027ukVziUGwVGYpG,spotify:track:0aB0v4027ukVziUGwVGYpG,https://api.spotify.com/v1/tracks/0aB0v4027ukV...,https://api.spotify.com/v1/audio-analysis/0aB0...,220690,4,United States
2,luther (with sza),"[2YZyLoL8N0Wb9xBt1NhZWg, 7tYKF4w9nC0nq9CsPZTHyP]",0.707,0.575,2,-7.546,1,0.1250,0.25100,0.000000,...,0.576,138.008,audio_features,2CGNAOSuO1MEFCbBRgUzjd,spotify:track:2CGNAOSuO1MEFCbBRgUzjd,https://api.spotify.com/v1/tracks/2CGNAOSuO1ME...,https://api.spotify.com/v1/audio-analysis/2CGN...,177599,4,United States
3,wacced out murals,[2YZyLoL8N0Wb9xBt1NhZWg],0.761,0.501,9,-10.759,0,0.4590,0.24900,0.000000,...,0.502,81.998,audio_features,5gOfC9UzZQzTyShqPMrpjT,spotify:track:5gOfC9UzZQzTyShqPMrpjT,https://api.spotify.com/v1/tracks/5gOfC9UzZQzT...,https://api.spotify.com/v1/audio-analysis/5gOf...,317093,4,United States
4,hey now (feat. dody6),"[2YZyLoL8N0Wb9xBt1NhZWg, 4VHa48wXlsDA2vWfgIi7cX]",0.806,0.358,10,-8.769,1,0.1170,0.00443,0.001620,...,0.188,102.011,audio_features,5S8VwnB4sLi6W0lYTWYylu,spotify:track:5S8VwnB4sLi6W0lYTWYylu,https://api.spotify.com/v1/tracks/5S8VwnB4sLi6...,https://api.spotify.com/v1/audio-analysis/5S8V...,217479,4,United States
5,reincarnated,[2YZyLoL8N0Wb9xBt1NhZWg],0.846,0.779,7,-5.363,1,0.2540,0.05340,0.000000,...,0.748,91.118,audio_features,0RgjEkSbeuStKfT2Pa4Zai,spotify:track:0RgjEkSbeuStKfT2Pa4Zai,https://api.spotify.com/v1/tracks/0RgjEkSbeuSt...,https://api.spotify.com/v1/audio-analysis/0Rgj...,275740,4,United States
6,"dodger blue (feat. wallie the sensei, siete7x,...","[2YZyLoL8N0Wb9xBt1NhZWg, 6cUpFVxDYWed9WxtC4QgC...",0.796,0.536,6,-7.930,1,0.1090,0.05520,0.000000,...,0.662,83.492,audio_features,4K1Pg0FLno1ltzX3jeqT83,spotify:track:4K1Pg0FLno1ltzX3jeqT83,https://api.spotify.com/v1/tracks/4K1Pg0FLno1l...,https://api.spotify.com/v1/audio-analysis/4K1P...,131241,4,United States
7,man at the garden,[2YZyLoL8N0Wb9xBt1NhZWg],0.817,0.348,7,-10.213,0,0.2900,0.51000,0.000119,...,0.634,91.997,audio_features,5ho7VSXSmI2KM2nDjcnLyz,spotify:track:5ho7VSXSmI2KM2nDjcnLyz,https://api.spotify.com/v1/tracks/5ho7VSXSmI2K...,https://api.spotify.com/v1/audio-analysis/5ho7...,233173,4,United States
8,That’s So True,[4tuJ0bMpJh08umKkEXKUI5],0.554,0.808,1,-4.169,1,0.0368,0.21400,0.000000,...,0.372,108.548,audio_features,7ne4VBA60CxGM75vw0EYad,spotify:track:7ne4VBA60CxGM75vw0EYad,https://api.spotify.com/v1/tracks/7ne4VBA60CxG...,https://api.spotify.com/v1/audio-analysis/7ne4...,166300,4,United States
9,"Sticky (feat. GloRilla, Sexyy Red & Lil Wayne)","[4V8LLVI7PbaPR0K2TGSxFF, 2qoQgPAilErOKCwE2Y8wO...",0.505,0.812,1,-3.986,1,0.4610,0.28500,0.000000,...,0.664,80.090,audio_features,3tFed7YsjGnIfxeLEQwx3R,spotify:track:3tFed7YsjGnIfxeLEQwx3R,https://api.spotify.com/v1/tracks/3tFed7YsjGnI...,https://api.spotify.com/v1/audio-analysis/3tFe...,255916,4,United States
